In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# import data
df1 = pd.read_csv('anonymous-msweb.data', sep=',', header=None, names=["a","b","c","d","e","f"])
df1.head(5)

C:\Users\nhuma\AppData\Local\Temp\ipykernel_16688\2982144896.py:7: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('anonymous-msweb.data', sep=',', header=None, names=["a","b","c","d","e","f"])


,a,b,c,d,e,f
0,I,4,www.microsoft.com,created by getlog.pl,NaN,NaN
1,T,1,VRoot,0,0,VRoot
2,N,0,0,NaN,NaN,NaN
3,N,1,1,NaN,NaN,NaN
4,T,2,Hide1,0,0,Hide


In [2]:
d = {}
with open("anonymous-msweb.data") as f:
    read = f.readlines()[7:301]
    for line in read:        
        (a, key, c,  val, e) = line.split(',')
        d[int(key)] = str(val.strip('"'))

print (d)

{1287: 'International AutoRoute', 1288: 'library', 1289: 'Master Chef Product Information', 1297: 'Central America', 1215: 'For Developers Only Info', 1279: 'Multimedia Golf', 1239: 'Microsoft Consulting', 1282: 'home', 1251: 'Reference Support', 1121: 'Microsoft Magazine', 1083: 'MS Access Support', 1145: 'Visual Fox Pro Support', 1276: 'Visual Test Support', 1200: 'Benelux Region', 1259: 'controls', 1155: 'Sidewalk', 1092: 'Visual FoxPro', 1004: 'Microsoft.com Search', 1057: 'MS PowerPoint News', 1140: 'Netherlands (Holland)', 1198: 'Picture It', 1147: 'Microsoft Financial Forum', 1005: 'Norway', 1026: 'Internet Site Construction for Developers', 1119: 'Corporation Information', 1216: 'Virtual Reality Markup Language', 1218: 'MS Publisher Support', 1205: 'Hardware Supprt', 1269: 'Customer Guides', 1031: 'MS Office', 1003: 'Knowledge Base', 1238: 'Excel Development', 1118: 'SQL Server', 1242: 'MS Garden', 1171: 'MS Merchant', 1175: 'MS Project Support', 1021: 'Visual C', 1222: 'MS Off

In [3]:
# data cleaning step

# replacing empty value with 0.
df1.fillna(0,inplace=True)

# prepare vroot_df and cv_df

# create new dataframe only storing values of A so quering is possible later
vroot_df = df1[df1.a == 'A']
vroot_df.astype({'b': 'string'}).dtypes

# create new dataframe with C and V lines so counting is possible for apriori algorithm
cv_df = df1.loc[(df1['a'] == 'C') | (df1['a'] == 'V')]
c_df = df1.loc[(df1['a'] == 'C')]
v_df = df1.loc[(df1['a'] == 'V')]

cv_df.head(5)

,a,b,c,d,e,f
301,C,10001,10001,0,0,0
302,V,1000,1,0,0,0
303,V,1001,1,0,0,0
304,V,1002,1,0,0,0
305,C,10002,10002,0,0,0


In [4]:
user_views = {}
userID = ""

feed_array = []

for index, row in cv_df.iterrows():
    if row['a'] == 'C':
        userID = row['b']
        user_views[userID] = []
    if row['a'] == 'V':
        inrow = []
        inrow.append(userID)
        inrow.append(row['b'])
        inrow.append(d[row['b']])
        inrow.append(1)
        np.transpose(inrow)

        feed_array.append(inrow)

# plan tmrw: import vroot_name as dictionary then look into how to use that info to build pivot_table


In [5]:
feed_array

[[10001, 1000, 'regwiz', 1],
 [10001, 1001, 'Support Desktop', 1],
 [10001, 1002, 'End User Produced View', 1],
 [10002, 1001, 'Support Desktop', 1],
 [10002, 1003, 'Knowledge Base', 1],
 [10003, 1001, 'Support Desktop', 1],
 [10003, 1003, 'Knowledge Base', 1],
 [10003, 1004, 'Microsoft.com Search', 1],
 [10004, 1005, 'Norway', 1],
 [10005, 1006, 'misc', 1],
 [10006, 1003, 'Knowledge Base', 1],
 [10006, 1004, 'Microsoft.com Search', 1],
 [10007, 1007, 'International IE content', 1],
 [10008, 1004, 'Microsoft.com Search', 1],
 [10009, 1008, 'Free Downloads', 1],
 [10009, 1009, 'Windows Family of OSs', 1],
 [10010, 1010, 'Visual Basic', 1],
 [10010, 1000, 'regwiz', 1],
 [10010, 1011, 'MS Office Development', 1],
 [10010, 1012, 'Outlook Development', 1],
 [10010, 1013, 'Visual Basic Support', 1],
 [10010, 1014, 'Office Free Stuff', 1],
 [10011, 1015, 'Excel', 1],
 [10011, 1016, 'MS Excel', 1],
 [10011, 1017, 'Products ', 1],
 [10011, 1018, 'isapi', 1],
 [10011, 1019, 'MS PowerPoint', 1],


In [6]:
# create df to serve as input for pivot df

pivot_feeder = pd.DataFrame(feed_array, columns=['userID', 'v_id', 'a_name', 'quantity'])
pivot_feeder

,userID,v_id,a_name,quantity
0,10001,1000,regwiz,1
1,10001,1001,Support Desktop,1
2,10001,1002,End User Produced View,1
3,10002,1001,Support Desktop,1
4,10002,1003,Knowledge Base,1
...,...,...,...,...
98649,42709,1003,Knowledge Base,1
98650,42710,1035,Windows95 Support,1
98651,42710,1001,Support Desktop,1
98652,42710,1018,isapi,1


In [7]:
# make the pivot table

pivot = pd.pivot_table(data=pivot_feeder,index='userID',columns='a_name', aggfunc='sum',values='quantity',fill_value=0)

In [10]:
def convert_into_binary(x):
    if x > 1:
        return 1
    else:
        return x

pivot = pivot.applymap(convert_into_binary)

pivot

a_name,About Microsoft,Access Development,ActiveX Data Objects,ActiveX Technology Development,Advanced Data Connector,Advanced Technology,Anti Piracy Information,Argentina,Australia,Authorized Technical Education Center Program,...,mdsn,misc,msdownload.,partner,promo,regwiz,security.,softlib,sports,vbscripts
userID,,,,,,,,,,,,,,,,,,,,,
10001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10005,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
frequent_itemsets = apriori(pivot, min_support=0.05, use_colnames=True)

c:\Users\nhuma\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [16]:
frequent_itemsets

,support,itemsets
0,0.331265,(Free Downloads)
1,0.286845,(Internet Explorer)
2,0.098438,(Internet Site Construction for Developers)
3,0.090734,(Knowledge Base)
4,0.258720,(Microsoft.com Search)
5,0.156155,(Products )
6,0.136070,(Support Desktop)
7,0.064902,(Web Site Builder's Gallery)
8,0.141481,(Windows Family of OSs)
9,0.054752,(Windows95 Support)


In [17]:
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Free Downloads),(Internet Explorer),0.331265,0.286845,0.160802,0.485419,1.692267,0.065780,1.385894
1,(Internet Explorer),(Free Downloads),0.286845,0.331265,0.160802,0.560588,1.692267,0.065780,1.521888
2,(Free Downloads),(Products ),0.331265,0.156155,0.061233,0.184847,1.183736,0.009504,1.035198
3,(Products ),(Free Downloads),0.156155,0.331265,0.061233,0.392130,1.183736,0.009504,1.100129
4,(Free Downloads),(Windows Family of OSs),0.331265,0.141481,0.077925,0.235234,1.662652,0.031057,1.122590
